* O(log(n)) Word-level Search:
* we have list of sentences sentences
* we want to search query word.

* Naive sentence-structure give us O(n) complexity
* can we search it in O(log(n)) time complexity?

In [11]:
# O(n) time complexity
sentences = ["I love cats", "I love dogs too"]

def search(sentences, query):
    results = []

    # For checking complexity
    n_operations = 0
    for sentence in sentences:
        n_operations += 1
        if query in sentence.lower():
            results.append(sentence)
    return results, n_operations

results, n_operations = search(sentences, 'cats')
print(f"n_operations: {n_operations} \nn_Sentences:{len(sentences)} \nresults: {results}") # ['I love cats']

n_operations: 2 
n_Sentences:2 
results: ['I love cats']


# Inverted Index:
* O(log(n)) time complexity
1. Create an inverted index
2. Sort the word lists in the inverted index
3. qUse binary search to find matching sentences

In [37]:
from collections import defaultdict
import bisect

class SentenceSearch:
    def __init__(self, sentences):
        self.sentences = sentences
        self.inverted_index = self._create_inverted_index()

    def _create_inverted_index(self):
        '''
        e.g.
        ```
        sentences = ["I love cats", "I love dogs too", "Cats are cute", "Dogs are loyal"]
        searcher = SentenceSearch(sentences)
        searcher.inverted_index
        
        # ----------------------------------
        #               Output:
        # ----------------------------------
            defaultdict(list,
                        {'i': [0, 1],
                        'love': [0, 1],
                        'cats': [0, 2],
                        'dogs': [1, 3],
                        'too': [1],
                        'are': [2, 3],
                        'cute': [2],
                        'loyal': [3]})
        ```
        '''
        
        
        inverted_index = defaultdict(list)
        for idx, sentence in enumerate(self.sentences):
            words = sentence.lower().split()
            for word in words:
                inverted_index[word].append(idx)
        
        # Sort the lists for each word
        for word in inverted_index:
            inverted_index[word].sort()
        
        return inverted_index

    def search(self, query):
        # Search sentences that contain all words in the query
        query_words = query.lower().split() # ['i', 'love']
        
        if not query_words:
            return []

        # Start with the first word's sentence indices
        result_indices = set(self.inverted_index.get(query_words[0], []))   # {0, 1}
        n_operations = 0


        # Intersect with other words' sentence indices
        for word in query_words[1:]:
            n_operations += 1
            word_indices = set(self.inverted_index.get(word, []))
            result_indices.intersection_update(word_indices)        

        return [self.sentences[idx] for idx in result_indices], n_operations

    def search2(self, query):
        # Search sentences that contain at least one word in the query
        '''
        # count number of words from query the sentence contains
        # counts 1 even if the sentence same contains query word twice. 
            e.g. 
                ```
                    sentence = "I love cats and I love cats"
                    query = "love"
                    retults = [{
                        sentence : "I love cats and I love cats",
                        score:1
                    }]
        
        e.g2.
        ```
        sentences = ["I love cats", "I love dogs too", "Cats are cute", "Dogs are loyal"]
        searcher = SentenceSearch(sentences)
        query = "I love cats and dogs"
        searcher.search(query)

        # Output:
        ([
            {
                sentence:'I love cats'
                score : 3
            },
            {
                sentence:'I love dogs too'
                score: 3
            },
            {
                sentence:'Cats are cute'
                score: 1
            },
            {
                sentence:'Dogs are loyal'
                score: 1
            }
            )
        ```
        '''
        query_words = query.lower().split()
        
        if not query_words:
            return []
        
        # Initialize a dictionary to store the score of each sentence
        sentence_scores = defaultdict(int)
        n_operations = 0

        # Count the number of words from the query the sentence contains
        for word in query_words:
            n_operations += 1
            for idx in self.inverted_index.get(word, []):
                sentence_scores[idx] += 1
        
        # Sort the sentences by score
        results = []
        for idx, score in sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True):
            results.append({
                'sentence': self.sentences[idx],
                'score': score
            })
        
        return results, n_operations


# Example usage
sentences = ["I love cats", "I love dogs too", "Cats are cute", "Dogs are loyal"]
searcher = SentenceSearch(sentences)

query = "I love"
results, n_operations = searcher.search(query)
print(f"Results for '{query}':", results, '\nn_operations:', n_operations)


query = "cats"
results, n_operations = searcher.search(query)
print(f"\nResults for '{query}':", results, '\nn_operations:', n_operations)

query = "I love cats and dogs"
results, n_operations = searcher.search2(query)
print(f"\nResults for '{query}':", results, '\nn_operations:', n_operations)



Results for 'I love': ['I love cats', 'I love dogs too'] 
n_operations: 1

Results for 'cats': ['I love cats', 'Cats are cute'] 
n_operations: 0

Results for 'I love cats and dogs': [{'sentence': 'I love cats', 'score': 3}, {'sentence': 'I love dogs too', 'score': 3}, {'sentence': 'Cats are cute', 'score': 1}, {'sentence': 'Dogs are loyal', 'score': 1}] 
n_operations: 5


In [12]:
searcher.inverted_index

defaultdict(list,
            {'i': [0, 1],
             'love': [0, 1],
             'cats': [0, 2],
             'dogs': [1, 3],
             'too': [1],
             'are': [2, 3],
             'cute': [2],
             'loyal': [3]})

In [28]:
query_words = "I love".lower().split()
query_words
result_indices = set(searcher.inverted_index.get(query_words[0], []))
result_indices

{0, 1}

In [29]:
set(searcher.inverted_index.get(query_words[1], []))

{0, 1}

# Mongo db built in full text search
* time complexity: O(log(n))
* Uses full-text search capabilities
* Performs stemming (reducing words to their root form)
* Ignores stop words (common words like "the", "a", "an")
* Supports phrase searches and negation